In [2]:
# 연산 처리  패키지
import pandas as pd
import numpy as np

# 시각화 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 통계분석 패키지
import statsmodels.api as sm

# 그래프 설정
%matplotlib inline

# 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 코랩 mount
from google.colab import drive
drive.mount('/content/drive')

# root_path = "C:/Users/user/Desktop/기계학습이해/"
root_path = "/content/drive/MyDrive/기계학습이해/"

# 데이터 읽어오기
df1 = pd.read_csv(root_path+"Dataset/(자동차보험) 고객별 사고 발생률 예측 모델링_1.csv", encoding = "cp949", engine='python')
df2 = pd.read_csv(root_path+"Dataset/(자동차보험) 고객별 사고 발생률 예측 모델링_2.csv", encoding = "cp949", engine='python')
df3 = pd.read_csv(root_path+"Dataset/(자동차보험) 고객별 사고 발생률 예측 모델링_3.csv", encoding = "cp949", engine='python')

Mounted at /content/drive


In [ ]:
# 데이터 이상 없는 df의 column 확인
col = df2.columns

# 오류로 생성된 column 삭제
df1 = df1[col]

# df1의 파일상 null 행 삭제
null_idx = df1[df1["차종"].isnull()].index.tolist()
df1 = df1.drop(index = null_idx)

df1

In [ ]:
# 데이터 프레임 합치기
df = pd.concat([df1, df2, df3], ignore_index = True)
df

In [ ]:
# 수치형인데 object인 column 형변환
df['YUHO'] = df['YUHO'].str.replace(',', '').astype(int)
df['SAGO'] = df['SAGO'].apply(lambda x: int(float(str(x).replace(',', '').split('.')[0])))
df['SAGO'] = df['SAGO'].replace(',', '').astype(int)

# column명 한글로 변환
df = df.rename(columns = {"ZINSRDAVL": "연령대", "ZIOSEXCD": "성별",
                        "ZDPRODSCD": "국산차량여부", "NCR": "직전3년간사고건수",
                        "ZCARPSGVL": "차량경과년수", "ZDRVLISCD___T": "운전자한정특별약관",
                        "ZENTCARCD": "가입경력코드", "ZCARISDAM": "차량가입금액",
                        "ZIMAGERVL": "영상기록장치특약가입", "YUHO": "유효대수",
                        "SAGO" : "사고건수"})

# 사용하지 않는 column drop
df = df.drop(columns = ['ZCPRLCLCD', 'ZDRVLISCD'])
df

In [ ]:
column_names = df.columns.tolist()
print(column_names)

In [ ]:
df['사고유무'] = df.apply(lambda row: 0 if row['사고건수'] == 0 else 1, axis = 1)

In [ ]:
# 0과 100을 제거하는 방법
df = df[(df['연령대'] != 0) & (df['연령대'] != 100)]
df['연령대'].unique()


In [ ]:
df = df[df['유효대수'] <= 1]
df['유효대수'].unique()

In [ ]:
# df = df[df['직전3년간사고건수'] <= 1]
df['직전3년간사고건수'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

In [ ]:
df

In [ ]:
import pandas as pd

In [ ]:
def make_df(df):
  # 연령대


  # 연령대 컬럼을 one-hot 인코딩으로 변환
  one_hot_age = pd.get_dummies(df['연령대']).reset_index(drop=True)

  # 은아님
  labeled_age = df['연령대'].apply(lambda x: int(str(x).replace(',', '').split('.')[0])).reset_index(drop=True)

  # 민호님
  column_age = df['연령대']
  numeric_age = column_age.replace({10: 19, 20: 25, 30: 35, 40: 44, 50: 54, 60: 64, 70: 73, 80: 83, 90: 93}).to_frame().reset_index(drop=True)

  # one-hot + 은아님
  one_hot_numeric_age = pd.concat([one_hot_age, numeric_age], axis = 1).reset_index(drop=True)

  # one_hot + 민호님
  labeled_numeric_age = pd.concat([labeled_age, numeric_age], axis = 1).reset_index(drop=True)

  ages = [one_hot_age, labeled_age, numeric_age,one_hot_numeric_age,labeled_numeric_age]


  # 직전 3년간 사고 건수

  one_hot_NCR = pd.get_dummies(df['직전3년간사고건수']).reset_index(drop=True)

  # 은아님
  # 신규 0, 무사고 1, 1회 2, 2회 3, 3회 4, 결측치 0 은 무사고로 편입
  labeled_NCR_1 = df['직전3년간사고건수'].apply(lambda x: str(x).replace('0', 'N'))
  labeled_NCR = labeled_NCR_1.apply(lambda x: int(str(x).replace('Z', '0').replace('N', '1').replace('D', '2').replace('C', '3').replace('B', '4'))).reset_index(drop=True)


  # 민호님
  column_NCR = df['직전3년간사고건수']
  numeric_NCR = column_NCR.replace({'N': 0, 'D': 1, 'C': 2, 'B': 3, 'Z': 0.15}).to_frame().reset_index(drop=True)

  # one-hot + 은아님
  one_hot_numeric_NCR = pd.concat([one_hot_NCR, numeric_NCR], axis = 1).reset_index(drop=True)

  # one_hot + 민호님
  labeled_numeric_NCR = pd.concat([labeled_NCR, numeric_NCR], axis = 1).reset_index(drop=True)

  NCRs = [one_hot_NCR, labeled_NCR, numeric_NCR, one_hot_numeric_NCR, labeled_numeric_NCR]


  ## 차량 경과년수

  one_hot_carAge = pd.get_dummies(df['차량경과년수']).reset_index(drop=True)

  # 은아님
  labeled_carAge = df['차량경과년수'].apply(lambda x: int(str(x).replace('신차', '0').replace('5년이하', '1').replace('10년이하', '2').replace('10년이상', '3'))).reset_index(drop=True)


  # 민호님
  column_carAge = df['차량경과년수']
  numeric_carAge = column_carAge.replace({'신차': 0, '5년이하': 3.16, '10년이하': 7.8, '10년이상': 12.86}).to_frame().reset_index(drop=True)

  # one-hot + 은아님
  one_hot_numeric_carAge = pd.concat([one_hot_carAge, numeric_carAge], axis = 1).reset_index(drop=True)


  # one_hot + 민호님
  labeled_numeric_carAge = pd.concat([labeled_carAge, numeric_carAge], axis = 1).reset_index(drop=True)

  carAges = [one_hot_carAge, labeled_carAge, numeric_carAge, one_hot_numeric_carAge, labeled_numeric_carAge]


  # 차종
  one_hot_carType = pd.get_dummies(df['차종']).reset_index(drop=True)

  # 은아님
  labeled_carType = df['차종'].apply(lambda x: int(str(x).replace('소형A', '0').replace('소형B', '1').replace('중형', '2').replace('대형', '3').replace('다목적1종','4').replace('다목적2종','5').replace('기타','6'))).reset_index(drop=True)


  # 민호님
  column_carType = df['차종']
  numeric_carType = column_carType.replace({'소형A': 998, '소형B':1500 , '중형': 1999, '대형': 2740,'다목적1종':2080, '다목적2종':2080 }).to_frame().reset_index(drop=True)

  # one-hot + 은아님
  one_hot_numeric_carType = pd.concat([one_hot_carType, numeric_carType], axis = 1).reset_index(drop=True)


  # one_hot + 민호님
  labeled_numeric_carType = pd.concat([labeled_carType, numeric_carType], axis = 1).reset_index(drop=True)


  carTypes = [one_hot_carType, labeled_carType, numeric_carType, one_hot_numeric_carType, labeled_numeric_carType]


  # 마일리지


  one_hot_mileage = pd.get_dummies(df['마일리지약정거리']).reset_index(drop=True)


  # 은아님
  labeled_mileage_1 = df['마일리지약정거리'].apply(lambda x: str(x).replace('12000K','4').replace('15000K','5').replace('미가입','6'))
  labeled_mileage = labeled_mileage_1.apply(lambda x: int(str(x).replace('3000K', '0').replace('5000K', '1').replace('7000K', '2').replace('10000K', '3'))).reset_index(drop=True)


  # 민호님
  column_mileage = df['마일리지약정거리']
  numeric_mileage = column_mileage.replace({'미가입': 0, '3000K': 3, '5000K': 5, '7000K': 7, '10000K':9 , '12000K':10, '15000K':15}).to_frame().reset_index(drop=True)

  # one-hot + 은아님
  one_hot_numeric_mileage = pd.concat([one_hot_mileage, numeric_mileage], axis = 1).reset_index(drop=True)

  # one_hot + 민호님
  labeled_numeric_mileage = pd.concat([labeled_mileage, numeric_mileage], axis = 1).reset_index(drop=True)

  mileages = [one_hot_mileage, labeled_mileage, numeric_mileage,one_hot_numeric_mileage, labeled_numeric_mileage]


  # 가입경력

  one_hot_exp = pd.get_dummies(df['가입경력코드']).reset_index(drop=True)

  # 은아님
  labeled_exp = df['가입경력코드'].apply(lambda x: int(str(x).split('.')[0])).reset_index(drop=True)

  exps = [one_hot_exp, labeled_exp]


  # 가입금액

  one_hot_money = pd.get_dummies(df['차량가입금액']).reset_index(drop=True)

  # 은아님
  labeled_money = df['차량가입금액'].apply(lambda x : int(str(x).replace('미가입','0').replace('5천만원이하','1').replace('1억이하','2').replace('1억이상','3'))).reset_index(drop=True)

  moneys = [one_hot_money, labeled_money]

  one_hot_sex = pd.get_dummies(df['성별'])
  one_hot_kor = pd.get_dummies(df['국산차량여부'])
  one_hot_rule = pd.get_dummies(df['운전자한정특별약관'])
  one_hot_cctv = pd.get_dummies(df['영상기록장치특약가입'])

  combined_df = pd.concat([one_hot_sex, one_hot_kor, one_hot_rule, one_hot_cctv], axis=1).reset_index(drop=True)

  df_y = df['사고유무'].reset_index(drop=True)

  return ages, NCRs, carAges, carTypes, mileages, exps, moneys, combined_df, df_y


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve, mean_squared_error, precision_score, recall_score
import math

In [ ]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

In [ ]:
# 여기 모델올거임

# 로지스틱 회귀 함수
def logistic_classifier(X_train, X_test, y_train, y_test):

    # LogisticRegression 초기화
    model = LogisticRegression()


    # 모델 학습
    model.fit(X_train, y_train)


    # 예측
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1].reshape(-1,1) # 양성 클래스에 대한 확률 추출


    # 평가지표
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    auc = roc_auc_score(y_test, y_proba)

    # 최적 threshold 값 & 그 때의 FPR 출력
    fper, tper, thresholds = roc_curve(y_test, y_proba)  # thresholds 별 fpr tpr 계산
    optimal_idx = np.argmax(tper - fper)                 # fpr, tpr 간 차이가 가장 클 때의 index 저장
    optimal_fpr = fper[optimal_idx]                      # 해당 index에 위치한 fpr값 저장
    optimal_threshold = thresholds[optimal_idx]

    # 최적 threshold로 진행할 때의 평가지표
    # y_optpred = (y_proba > thresholds[optimal_idx]).astype(int)
    # accuracy = accuracy_score(y_test, y_optpred)
    # precision = precision_score(y_test, y_optpred)
    # recall = recall_score(y_test, y_optpred)
    # f1 = f1_score(y_test, y_optpred, average='weighted')



    return [accuracy, precision, recall, f1, auc, optimal_fpr]



In [ ]:
import pandas as pd

result_dir = root_path + "/result/things.csv"
result_df = pd.DataFrame(columns=["name", "accuracy", "precision", "recall", "f1", "auc", "optimal_fpr"])


result = dict()

for _ in range(30):

  ## 8천개를 먼저 뽑고
  now_df = df.sample(n=8000)

  # 모든 라벨링 경우의 수를 만듬
  ages, NCRs, carAges, carTypes, mileages, exps, moneys, combined_df, df_y = make_df(now_df)

  count = 0
  for a, age in enumerate(ages, start=1):
    for b, NCR in enumerate(NCRs, start=1):
      for c, carAge in enumerate(carAges, start=1):
        for d, carType in enumerate(carTypes, start=1):
          for e, mileage in enumerate(mileages, start=1):
            for f, exp in enumerate(exps, start=1):
              for g, money in enumerate(moneys, start=1):

                name = str(a) + str(b) + str(c) + str(d) + str(e) + str(f) + str(g)


                comb_df = pd.concat([combined_df, age, NCR, carAge, carType, mileage,exp, money], axis=1)

                comb_df.columns = comb_df.columns.astype(str)


                comb_df_numeric = comb_df.astype(int)


                X_train, X_test, y_train, y_test = train_test_split(comb_df_numeric, df_y, test_size = 0.2)


                # SMOTE를 이용하여 증강
                smote = SMOTE(sampling_strategy='auto')  # 'auto'는 소수 클래스의 샘플 수를 맞춥니다.
                X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

                accuracy, precision, recall, f1, auc, optimal_fpr = logistic_classifier(X_train_resampled, X_test, y_train_resampled, y_test)


                if name not in result:
                  result[name] = [[accuracy], [precision], [recall], [f1], [auc], [optimal_fpr]]
                else:
                  result[name][0].append(accuracy)
                  result[name][1].append(precision)
                  result[name][2].append(recall)
                  result[name][3].append(f1)
                  result[name][4].append(auc)
                  result[name][5].append(optimal_fpr)

  # result_df.to_csv(result_dir, index=False)
  print(i/30, "% 완료")




for key, value in result.items():
        for i in range(len(value)):
            value[i] = np.median(value[i])


# DataFrame으로 변환
df_result = pd.DataFrame.from_dict(result, columns=['accuracy', 'precision', 'recall', 'f1', 'auc', 'optimal_fpr'])

df_result.to_csv(result_dir, index=False)


## 컬럼네임 해석

age, NCR, carAge, carType, mileage,exp, money 순서대로

1: one-hot

2: 은아님

3: 민호님

4: one-hot + 은아님

5: one-hot + 민호님



  예시로 111112 은 전부 one-hot이고, money만 은아님 방식으로 라벨링
